# H-2 Certification Filtering

This notebook finds, filters, and calculates the number of workers certified for various employers by the Department of Labor's Office of Foreign Labor Certification. For more details, see [the main project page](https://github.com/BuzzFeedNews/2016-05-H-2-debarments-and-violations).

## Load the OFLC certification data

In [1]:
import pandas as pd

In [2]:
decisions = pd.read_csv(
    "../data/H-2-certification-decisions.csv",
    parse_dates=["last_event_date"],
    low_memory=False
).set_index("case_no")

The columns in that dataset:

In [3]:
decisions.dtypes

visa_type                           object
fy                                   int64
last_event_date             datetime64[ns]
case_status                         object
n_requested                        float64
n_certified                          int64
is_certified                          bool
certification_begin_date            object
certification_end_date              object
job_title                           object
employer_name                       object
employer_state                      object
employer_city                       object
employer_address_1                  object
employer_address_2                  object
employer_postal_code                object
worksite_state                      object
worksite_city                       object
agent_name                          object
organization_flag                   object
is_duplicate                        object
dtype: object

In [4]:
MAIN_COLUMNS = [
    "employer_name",
    "employer_city",
    "employer_state",
    "case_status",
    "last_event_date",
    "n_certified"
]

## Popsy Pop certifications since Nov. 11, 2012

In [5]:
popsy_pop = decisions[
    decisions["employer_name"].str.contains(r"POPSY POP", na=False) &
    (decisions["last_event_date"] > "2012-11-02")
][MAIN_COLUMNS]
popsy_pop.to_csv("../data/certification-lists/popsy-pop-since-2012-11-02.csv")
popsy_pop["n_certified"].sum()

106

Additionally, since the DOL's latest release of quarterly data, Popsy Pop has received certification for 34 workers, based on these two applications:

- `H-400-15323-974876`: 14 workers, certified on 2016-01-12: https://icert.doleta.gov/index.cfm?event=ehLCJRExternal.dspCert&doc_id=3&visa_class_id=7&id=84153
- `H-400-15348-000984`: 20 workers, certified on 2016-03-07: https://icert.doleta.gov/index.cfm?event=ehLCJRExternal.dspCert&doc_id=3&visa_class_id=7&id=87838


## National Agricultural Consultants

In [6]:
nat_ag = decisions[
    decisions["agent_name"].str.contains(r"\bNATIONAL AG.* CONSULT", na=False) &
    (decisions["last_event_date"] >= "2014-08-03") &
    (decisions["is_duplicate"] != True)
][MAIN_COLUMNS + ["agent_name"]]
nat_ag.to_csv("../data/certification-lists/national-agricultural-consultants-since-2014-08-03.csv")
nat_ag.groupby("agent_name")["n_certified"].sum()

agent_name
NATIONAL AGRICULTURAL CONSULTANTS LLC:THERESA WARD     5388
NATIONAL AGRICULTURAL CONSULTANTS LLC:WARREN WICKER      52
NATIONAL AGRICULTURAL CONSULTANTS:THERESA WARD           15
NATIONAL AGRICULTURE CONSULTANTS LLC:THERESA WARD         0
NATIONAL AGRICULURAL CONSULTANTS LLC:THERESA WARD        12
Name: n_certified, dtype: int64

In [7]:
nat_ag["n_certified"].sum()

5467

## T. Bell Detasseling

*Note: The earliest OFLC data for the H-2A program does not include specific dates of certification, only fiscal years, so the analysis below examines fiscal year 2006 (which began Oct. 1, 2005) through the end of calendar year 2015.*

In [8]:
t_bell = decisions[
    decisions["employer_name"].str.contains(r"BELL.* DETAS", na=False) &
    (decisions["fy"] >= 2006) &
    ((decisions["fy"] <= 2015) | (decisions["last_event_date"].dt.year == 2015))
][MAIN_COLUMNS]
t_bell.to_csv("../data/certification-lists/t-bell-fy-2006-through-cy-2015.csv")
t_bell.groupby("employer_name")["n_certified"].sum()

employer_name
BELL'S MELONS & TOM BELLS DETASSELING                           110
BELLS MELONS & TOM BELLS DETASSELING #1                         120
BELLS MELONS & TOM BELLS DETASSELING #2                          68
BELLS MEONS/ TOM BELLS DETASSELING #1                            45
T BELL DETASSELING LLC                                         1334
T.  BELL DETASSELING LLC                                         40
T. BELL DETASSELING LLC                                        2572
T. BELL DETASSELING LLC #1                                       16
T. BELL DETASSELING LLC #2                                      689
T. BELL DETASSELING LLC #3                                      307
T. BELL DETASSLEING LLC                                         616
T. BELL DETASSLEING LLC #2                                      696
TOM & TERI BELLS DBA: BELLS MELONS AND TOM BELL DETASSELING       4
TOM & TERRI BELL DBA BELLS MELONS & TOM BELL DETASSELING        192
TOM BELL DETASSELING              

In [9]:
t_bell["n_certified"].sum()

6869

### Certifications since Jan. 30, 2015

... "the date investigators submitted their final report"

In [10]:
t_bell_since_20150130 = decisions[
    decisions["employer_name"].str.contains(r"BELL.* DETAS", na=False) &
    (decisions["last_event_date"] > "2015-01-30")
][MAIN_COLUMNS]
t_bell_since_20150130.to_csv("../data/certification-lists/t-bell-since-2015-01-30.csv")
t_bell_since_20150130.groupby("employer_name")["n_certified"].sum()

employer_name
T. BELL DETASSELING LLC    473
Name: n_certified, dtype: int64

## Vasquez Citrus

In [11]:
vasquez_citrus = decisions[
    decisions["employer_name"].str.contains(r"VASQUEZ.* CITRUS|ROBERTO VASQUEZ|JUAN VASQUEZ", na=False)
][MAIN_COLUMNS]
vasquez_citrus.to_csv("../data/certification-lists/vasquez-citrus.csv")
vasquez_citrus.groupby("employer_name")["n_certified"].sum()

employer_name
JUAN VASQUEZ                      302
VASQUEZ CITRUS & HAULING, INC.    451
VASQUEZ CITRUS AND HAULING          0
Name: n_certified, dtype: int64

Additionally, since the DOL's latest release of quarterly data, Vasquez Citrus, Juan Vasquez, and/or Roberto Vasquez have received certification for 390 workers, based on these three applications:

- `H-300-15338-273530`: 49 workers, certified on 2016-01-04: https://icert.doleta.gov/index.cfm?event=ehLCJRExternal.dspCert&doc_id=3&visa_class_id=8&id=84771
- `H-300-16004-246335`: 225 workers, certified on 2016-01-20: https://icert.doleta.gov/index.cfm?event=ehLCJRExternal.dspCert&doc_id=3&visa_class_id=8&id=86603
- `H-300-16081-864237`: 116 workers, certified on 2016-04-26: https://icert.doleta.gov/index.cfm?event=ehLCJRExternal.dspCert&doc_id=3&visa_class_id=8&id=93175

---

---

---